<a href="https://colab.research.google.com/github/CMallart/ateliers-NN/blob/main/Creation_de_paysages_par_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data from :
https://www.kaggle.com/arnaud58/landscape-pictures?select=00000001_%285%29.jpg

# Generative adversarial networks

Les réseaux de neurones peu profonds sont somme toute des versions un peu compliquées de classifieurs statistiques, utilisés le plus souvent pour classifier des objets (phrases, images, audio, videos...) en plusieurs catégories.Il faut leur créer des variables(features) à la main et éspérer que cela leur parle.

Mais les réseaux de neurones profonds, eux, permettent de faire quelque chose de beaucoup plus puissante : ils apprennent des représentations. Ils peuvent apprendre directement, sans création de features à la main, des représentations par matrices d'objets complexes, comme des images. Cela serait dommage de ne pas exploiter cela pour quelque chose de plus puissant, ou au moins de différent.

Entrent en jeu les réseaux génératifs. Ces réseaux profitent du fait qu'on peut apprendre à un réseau à créer une représentation abstraite, vectorielle, d'un objet. 


In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
! git clone https://github.com/CMallart/ateliers-NN.git

In [ ]:
! ls

In [ ]:
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
import numpy as np
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot

In [ ]:
# load a generator for visualisation first
visual_generator = ImageDataGenerator(fill_mode="reflect",
                                     horizontal_flip=True)
visual_flow = visual_generator.flow_from_directory("/content/ateliers-NN/data/landscapes", class_mode=None, target_size=(256,256), batch_size=1, shuffle=False)


images= []
for i in range(0,4):
  images.append(visual_flow.next()[0])

print(images[0].shape, images[0])

plt.figure(figsize=(10, 10))
plt.subplot(221)
plt.imshow((images[0]).astype('uint8'))

plt.subplot(222)
plt.imshow((images[1]).astype('uint8'))

plt.subplot(223)
plt.imshow((images[2]).astype('uint8'))

plt.subplot(224)
plt.imshow((images[3]).astype('uint8'))

plt.show()

In [ ]:
def define_discriminator(in_shape=(128,128,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [ ]:
def define_discriminator(in_shape=(128,128,3)):
  model = Sequential()
  model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=in_shape, padding="same"))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
  model.add(ZeroPadding2D(padding=((0,1),(0,1))))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))

In [ ]:
# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 8x8 image
	n_nodes = 256 * 8 * 8
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((8, 8, 256)))
	# upsample to 16x16
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 32x32
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 64x64
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
 	# upsample to 128x128
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# output layer
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

In [ ]:
def build_generator(self):

    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

In [70]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [77]:
# to train the discriminator, we need a mix of real and fake images

def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y

# use the generator to generate n fake examples, with class labels 0
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y


# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

When training the generator via this logical GAN model, there is one more important change. We want the discriminator to think that the samples output by the generator are real, not fake. Therefore, when the generator is trained as part of the GAN model, we will mark the generated samples as real (class 1).

Why would we want to do this?

We can imagine that the discriminator will then classify the generated samples as not real (class 0) or a low probability of being real (0.3 or 0.5). The backpropagation process used to update the model weights will see this as a large error and will update the model weights (i.e. only the weights in the generator) to correct for this error, in turn making the generator better at generating good fake samples.

Let’s make this concrete.

Inputs: Point in latent space, e.g. a 100-element vector of Gaussian random numbers.
Outputs: Binary classification, likelihood the sample is real (or fake).
The define_gan() function below takes as arguments the already-defined generator and discriminator models and creates the new, logical third model subsuming these two models. The weights in the discriminator are marked as not trainable, which only affects the weights as seen by the GAN model and not the standalone discriminator model.

The GAN model then uses the same binary cross entropy loss function as the discriminator and the efficient Adam version of stochastic gradient descent with the learning rate of 0.0002 and momentum of 0.5, recommended when training deep convolutional GANs.

In [97]:
from google.colab import files
import os

folder_saves = "/content/drive/My Drive/Colab_Notebooks/plots_gan/"
if not os.path.exists(folder_saves):
    os.makedirs(folder_saves)

# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	plt.figure(figsize=(40, 40))
	# plot images
	for i in range(n * n):
		# define subplot
		plt.subplot(n, n, 1 + i)
		# turn off axis
		plt.axis('off')
		# plot raw pixel data
		plt.imshow(examples[i])
	# save plot to file
	filename = folder_saves+'generated_plot_e%03d.png' % (epoch+1)
	plt.savefig(filename)
	files.download(filename) 

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = folder_saves+'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)
	files.download(filename)

In [91]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=30, n_batch=128):
	bat_per_epo = int(4335 / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			print('>Epoch :%d, Batch : %d/%d, discrim_real_loss=%.3f, discrim_fake_loss=%.3f gan_loss=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim, n_samples=256)

In [41]:
train_generator = ImageDataGenerator(rescale=1/255,fill_mode="reflect")
train_flow = train_generator.flow_from_directory("/content/ateliers-NN/data/landscapes", class_mode=None, target_size=(128,128), batch_size=4319)

Found 4319 images belonging to 1 classes.


In [42]:
dataset_train = []
print("Starting")
temp_data = train_flow.next()
for i in range(temp_data.shape[0]):
  dataset_train.append(temp_data[i])
print("Done")

Starting


In [51]:
dataset = np.array(dataset_train)

In [92]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)

In [93]:
d_model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 128, 128, 64)      1792      
_________________________________________________________________
leaky_re_lu_69 (LeakyReLU)   (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 64, 64, 128)       73856     
_________________________________________________________________
leaky_re_lu_70 (LeakyReLU)   (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 32, 32, 128)       147584    
_________________________________________________________________
leaky_re_lu_71 (LeakyReLU)   (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 16, 16, 256)     

In [94]:
g_model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 16384)             1654784   
_________________________________________________________________
leaky_re_lu_73 (LeakyReLU)   (None, 16384)             0         
_________________________________________________________________
reshape_7 (Reshape)          (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_34 (Conv2DT (None, 16, 16, 128)       524416    
_________________________________________________________________
leaky_re_lu_74 (LeakyReLU)   (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_35 (Conv2DT (None, 32, 32, 128)       262272    
_________________________________________________________________
leaky_re_lu_75 (LeakyReLU)   (None, 32, 32, 128)     

In [95]:
gan_model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_22 (Sequential)   (None, 128, 128, 3)       2969475   
_________________________________________________________________
sequential_21 (Sequential)   (None, 1)                 583937    
Total params: 3,553,412
Trainable params: 2,969,475
Non-trainable params: 583,937
_________________________________________________________________


In [ ]:
# train model
train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch = 256)

>Epoch :1, Batch : 1/16, discrim_real_loss=0.566, discrim_fake_loss=0.561 gan_loss=1.160
>Epoch :1, Batch : 2/16, discrim_real_loss=0.471, discrim_fake_loss=0.537 gan_loss=1.262
>Epoch :1, Batch : 3/16, discrim_real_loss=0.456, discrim_fake_loss=0.478 gan_loss=1.344
>Epoch :1, Batch : 4/16, discrim_real_loss=0.434, discrim_fake_loss=0.461 gan_loss=1.429
>Epoch :1, Batch : 5/16, discrim_real_loss=0.401, discrim_fake_loss=0.527 gan_loss=1.648
>Epoch :1, Batch : 6/16, discrim_real_loss=0.365, discrim_fake_loss=0.423 gan_loss=1.747
>Epoch :1, Batch : 7/16, discrim_real_loss=0.330, discrim_fake_loss=0.611 gan_loss=1.786
>Epoch :1, Batch : 8/16, discrim_real_loss=0.633, discrim_fake_loss=1.267 gan_loss=1.672
>Epoch :1, Batch : 9/16, discrim_real_loss=1.006, discrim_fake_loss=0.776 gan_loss=1.481
>Epoch :1, Batch : 10/16, discrim_real_loss=0.978, discrim_fake_loss=0.611 gan_loss=1.431
>Epoch :1, Batch : 11/16, discrim_real_loss=0.914, discrim_fake_loss=0.469 gan_loss=1.464
>Epoch :1, Batch : 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

>Epoch :91, Batch : 1/16, discrim_real_loss=0.506, discrim_fake_loss=0.499 gan_loss=1.896
>Epoch :91, Batch : 2/16, discrim_real_loss=0.686, discrim_fake_loss=0.623 gan_loss=1.776
>Epoch :91, Batch : 3/16, discrim_real_loss=0.674, discrim_fake_loss=0.492 gan_loss=1.772
>Epoch :91, Batch : 4/16, discrim_real_loss=0.648, discrim_fake_loss=0.483 gan_loss=1.561
>Epoch :91, Batch : 5/16, discrim_real_loss=0.572, discrim_fake_loss=0.525 gan_loss=1.420
>Epoch :91, Batch : 6/16, discrim_real_loss=0.474, discrim_fake_loss=0.597 gan_loss=1.537
>Epoch :91, Batch : 7/16, discrim_real_loss=0.509, discrim_fake_loss=0.455 gan_loss=1.562
>Epoch :91, Batch : 8/16, discrim_real_loss=0.490, discrim_fake_loss=0.586 gan_loss=1.649
>Epoch :91, Batch : 9/16, discrim_real_loss=0.529, discrim_fake_loss=0.412 gan_loss=1.757
>Epoch :91, Batch : 10/16, discrim_real_loss=0.596, discrim_fake_loss=0.570 gan_loss=1.894
>Epoch :91, Batch : 11/16, discrim_real_loss=0.612, discrim_fake_loss=0.472 gan_loss=1.948
>Epoch :

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>